#### More hashing examples
- More examples to becomes as familiar as possible
<blockquote>
<p>Example 1: <a href="https://leetcode.com/problems/group-anagrams/" target="_blank">49. Group Anagrams</a></p>
<p>Given an array of strings <code>strs</code>, group the <a href="https://en.wikipedia.org/wiki/Anagram" target="_blank">anagrams</a> together.</p>
<p>For example, given <code>strs = ["eat","tea","tan","ate","nat","bat"]</code>, return <code>[["bat"],["nat","tan"],["ate","eat","tea"]]</code>.</p>
</blockquote>

1) Method 1: <u>Check if the two strings are equal after both being sorted</u>
    - Use the sorted version as a key (**all anagrams are the same once they are sorted in alphabetical order**)
    - Map these keys to the groups *(Ex: ["ate","eat","tea"])* themselves
    - Answer just becomes the values of the hash map

In [8]:
from collections import defaultdict
from typing import List
from pprint import pprint

def groupAnagrams(strs: List[str]) -> List[List[str]]:
    groups = defaultdict(list)
    for s in strs:
        key: str = "".join(sorted(s))
        groups[key].append(s)
    
    pprint(groups)
        
    return list(groups.values())

In [9]:
strs = ["eat","tea","tan","ate","nat","bat"]
groupAnagrams(strs)

defaultdict(<class 'list'>,
            {'abt': ['bat'],
             'aet': ['eat', 'tea', 'ate'],
             'ant': ['tan', 'nat']})


[['eat', 'tea', 'ate'], ['tan', 'nat'], ['bat']]

<blockquote>
<p>Another way to solve this problem is to use a tuple of length 26 representing the count of each character as the key instead of the sorted string. This would technically solve the problem in <span class="maths katex-rendered"><span class="katex"><span class="katex-mathml"><math><semantics><mrow><mi>O</mi><mo>(</mo><mi>n</mi><mo>⋅</mo><mi>m</mi><mo>)</mo></mrow><annotation encoding="application/x-tex">O(n \cdot m)</annotation></semantics></math></span><span class="katex-html" aria-hidden="true"><span class="base"><span class="strut" style="height: 1em; vertical-align: -0.25em;"></span><span class="mord mathdefault" style="margin-right: 0.02778em;">O</span><span class="mopen">(</span><span class="mord mathdefault">n</span><span class="mspace" style="margin-right: 0.222222em;"></span><span class="mbin">⋅</span><span class="mspace" style="margin-right: 0.222222em;"></span></span><span class="base"><span class="strut" style="height: 1em; vertical-align: -0.25em;"></span><span class="mord mathdefault">m</span><span class="mclose">)</span></span></span></span></span> because the 26 is a constant defined by the problem, but for test cases with smaller strings it would be slower due to the constant factor which is hidden by big O.</p>
<p>It also assumes that the strings can only have 26 different characters, which is valid here but less general and less resistant to follow-ups.</p>
</blockquote>
<hr>
<blockquote>
<p>Example 2: <a href="https://leetcode.com/problems/minimum-consecutive-cards-to-pick-up/" target="_blank">2260. Minimum Consecutive Cards to Pick Up</a></p>
<p>Given an integer array <code>cards</code>, find the length of the shortest subarray that contains at least one duplicate. If the array has no duplicates, return <code>-1</code>.</p>
</blockquote>
<p>We can actually solve this problem using a sliding window, but let's take a look at a different approach that has more emphasis on a hash map. This question is equivalent to: what is the shortest distance between any two of the same element? If we go through the array and use a hash map to record the indices for every element, we can iterate over those indices to find the shortest distance. For example, given <code>cards = [1, 2, 6, 2, 1]</code>, we would map <code>1: [0, 4]</code>, <code>2: [1, 3]</code>, and <code>6: [2]</code>. Then we can iterate over the values and see that the minimum difference can be achieved from picking up the <code>2</code>s.</p>


In [17]:
from collections import defaultdict

def minimumCardPickup(cards: List[int]) -> int:
    dic = defaultdict(list)
    for i in range(len(cards)):
        dic[cards[i]].append(i)         # For each card, the key is the card's value (i.e. [1, 2, ...]) and the value is its index
    
    ans = float("inf")                  # Initialize the answer to infinity
    for key in dic:
        arr = dic[key]                  # arr is a list of indices for the current card value (ex: {1: [0, 4]})
        for i in range(len(arr) -1):
            ans = min(ans, arr[i + 1] - arr[i] + 1)
            
    pprint(dic)
        
    return ans if ans < float("inf") else -1

In [18]:
cards = [1, 2, 6, 2, 1]
minimumCardPickup(cards)

defaultdict(<class 'list'>, {1: [0, 4], 2: [1, 3], 6: [2]})


3

<p>We can actually improve this algorithm slightly by observing that we don't need to store all the indices, but only the most recent one that we saw for each number. This improves the average space complexity. The current algorithm has <span class="maths katex-rendered"><span class="katex"><span class="katex-mathml"><math><semantics><mrow><mi>O</mi><mo>(</mo><mi>n</mi><mo>)</mo></mrow><annotation encoding="application/x-tex">O(n)</annotation></semantics></math></span><span class="katex-html" aria-hidden="true"><span class="base"><span class="strut" style="height: 1em; vertical-align: -0.25em;"></span><span class="mord mathdefault" style="margin-right: 0.02778em;">O</span><span class="mopen">(</span><span class="mord mathdefault">n</span><span class="mclose">)</span></span></span></span></span> space complexity always, but with the improvement, it is only <span class="maths katex-rendered"><span class="katex"><span class="katex-mathml"><math><semantics><mrow><mi>O</mi><mo>(</mo><mi>n</mi><mo>)</mo></mrow><annotation encoding="application/x-tex">O(n)</annotation></semantics></math></span><span class="katex-html" aria-hidden="true"><span class="base"><span class="strut" style="height: 1em; vertical-align: -0.25em;"></span><span class="mord mathdefault" style="margin-right: 0.02778em;">O</span><span class="mopen">(</span><span class="mord mathdefault">n</span><span class="mclose">)</span></span></span></span></span> in the worst case, when there are no duplicates.</p>

In [22]:
from collections import defaultdict

class Solution:
    def minimumCardPickup(self, cards: List[int]) -> int:
        dic = defaultdict(int)
        ans = float("inf")
        for i in range(len(cards)):
            # When we see a card we've seen before:
            if cards[i] in dic:
                ans = min(ans, i - dic[cards[i]] + 1)  # Distance from most recent occurrence
            # Always update to current position (overwrites old position):
            dic[cards[i]] = i
        pprint(dic)
        return ans if ans < float("inf") else -1
    
    # Literally the exact same thing as before

In [21]:
cards = [3, 4, 2, 3, 4, 7]
minimumCardPickup(cards)

defaultdict(<class 'list'>, {3: [0, 3], 4: [1, 4], 2: [2], 7: [5]})


4

<hr>
<blockquote>
<p>Example 3: <a href="https://leetcode.com/problems/max-sum-of-a-pair-with-equal-sum-of-digits/" target="_blank">2342. Max Sum of a Pair With Equal Sum of Digits</a></p>
<p>Given an array of integers <code>nums</code>, find the maximum value of <code>nums[i] + nums[j]</code>, where <code>nums[i]</code> and <code>nums[j]</code> have the same <strong>digit sum</strong> (the sum of their individual digits). Return <code>-1</code> if there is no pair of numbers with the same digit sum.</p>
</blockquote>

In [37]:
from collections import defaultdict

class Solution:
    def maximumSum(self, nums: List[int]) -> int:
        def get_digit_sum(num):
            """
            Helper function to calculate the sum of digits of a number in the list 
            """
            digit_sum = 0
            while num:                          # While there are still digits left
                digit_sum += num % 10           # Get the ones-digit
                num //=  10                     # Get the tens digit
            return digit_sum

        dic = defaultdict(list)
        for num in nums:
            digit_sum = get_digit_sum(num)
            dic[digit_sum].append(num)              # if (ex) nums = [18, 36] // "dic[digit_sum] == 9" && ".append(num) == [18, 36]"
        pprint(dic)

        ans = -1
        for key in dic:
            curr = dic[key]             # current is the value of the key in the dictionary (ex: in {1: [1, 10]}, current is [1, 10]) & key is the sum of digits
            pprint(curr)
            if len(curr) > 1:
                curr.sort(reverse=True)
                ans = max(ans, curr[0] + curr[1])
                
        return ans

In [38]:
nums = [18,43,36,13,7]
sol = Solution()
sol.maximumSum(nums)

defaultdict(<class 'list'>, {9: [18, 36], 7: [43, 7], 4: [13]})
[18, 36]
[43, 7]
[13]


54

- implementing the same method as before by only saving the largest number seen so far for each digit sum

In [ ]:
from collections import defaultdict

class Solution:
    def maximumSum(self, nums: List[int]) -> int:
        def get_digit_sum(num):
            digit_sum = 0
            while num:
                digit_sum += num % 10
                num //= 10
            
            return digit_sum
        
        # Ex: with nums = [18,43,36,13,7]
        dic = defaultdict(int)
        ans = -1
        for num in nums:
            digit_sum = get_digit_sum(num)
            if digit_sum in dic:                                # If I have already seen another number ex: [18] with the same digit sum (ex: 9)
                ans = max(ans, num + dic[digit_sum])            # Add the value of this number to the value that has the same digit sum (ex: {9:[18,...,36]} == replace previous ans with (18 + 36))

            dic[digit_sum] = max(dic[digit_sum], num)       # Ex: dic[9] = max(dic[9], 36) → max(18, 36) = 36

        return ans

<blockquote>
<p>Example 4: <a href="https://leetcode.com/problems/equal-row-and-column-pairs/" target="_blank">2352. Equal Row and Column Pairs</a></p>
<p>Given an <code>n x n</code> matrix <code>grid</code>, return the number of pairs <code>(R, C)</code> where <code>R</code> is a row and <code>C</code> is a column, and <code>R</code> and <code>C</code> are equal if we consider them as 1D arrays.</p>
</blockquote>
<hr>

**Ex:** grid = [[3,2,1],
                [1,7,6],
                [2,7,7]]
        Output = 1 because <u>Row 2 is [2,7,7] and Column 2 is [2],[7],[7]</u>
<hr>

1) Use a hash map to count how many times each row occurs
2) Use a second has map to do the same thing with columns
3) Iterate over the **rows hash map** and check **for each row**, check if the same array appeared as a **column**
4) If yes, Then the product of the # of appearances is added as an answer
5) Because we cannot use an `array` as a key (they are mutable), we should use either a `string` or a `tuple`

In [49]:
from collections import defaultdict

class Solution:
    def equalPairs(self, grid: List[List[int]]) -> int:
        def convert_to_key(arr):
            return tuple(arr)
        
        # Row Dictionary
        dic = defaultdict(int)
        for row in grid:
            dic[convert_to_key(row)] += 1               # Key = the tuple of the row-arr, value == # of times it has occurred
        
        # Column Dictionary
        dic2 = defaultdict(int)
        for col in range(len(grid[0])):                 # len(grid[0]) is the index of the first row, which is the number of columns
            current_col = []
            for row in range(len(grid)):
                current_col.append(grid[row][col])
                
            dic2[convert_to_key(current_col)] += 1
        pprint(dic2)
        ans = 0
        for arr in dic:
            ans += dic[arr] * dic2[arr]                 # Ex: if there were 2 rows that had the same value, and 2 columns, there would be 4 possible combos
            
        return ans

In [51]:
grid = [[3,2,1],
        [1,7,6],
        [2,7,7]]

sol = Solution()
sol.equalPairs(grid)

defaultdict(<class 'int'>, {(3, 1, 2): 1, (2, 7, 7): 1, (1, 6, 7): 1})


1